In [1]:
# pip install qiskit==0.31.0

In [1]:
from get_test_datasets import return_dataset
test_dataset = return_dataset()

Generating QFT circuits
Generating Grover circuits


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 868/868 [00:01<00:00, 501.41it/s]


Generating QPE circuits


In [2]:
len(test_dataset)

1338

In [5]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from helper import *

ibmqfactory.load_account:WARNING:2023-11-27 00:46:08,838: Credentials are already in use. The existing account in the session will be replaced.


{'ibm_brisbane': [[1, 0], [2, 1], [3, 2], [4, 3], [4, 5], [4, 15], [6, 5], [6, 7], [7, 8], [8, 9], [10, 9], [10, 11], [11, 12], [12, 17], [13, 12], [14, 0], [14, 18], [15, 22], [16, 8], [16, 26], [17, 30], [18, 19], [20, 19], [20, 33], [21, 20], [21, 22], [22, 23], [24, 23], [24, 34], [25, 24], [26, 25], [27, 26], [28, 27], [28, 29], [28, 35], [30, 29], [30, 31], [31, 32], [32, 36], [33, 39], [34, 43], [35, 47], [36, 51], [37, 38], [39, 38], [40, 39], [40, 41], [41, 53], [42, 41], [42, 43], [43, 44], [44, 45], [46, 45], [46, 47], [48, 47], [48, 49], [50, 49], [50, 51], [52, 37], [52, 56], [53, 60], [54, 45], [54, 64], [55, 49], [55, 68], [56, 57], [57, 58], [58, 59], [58, 71], [59, 60], [60, 61], [62, 61], [62, 63], [62, 72], [63, 64], [65, 64], [65, 66], [67, 66], [67, 68], [69, 68], [69, 70], [73, 66], [74, 70], [74, 89], [75, 90], [76, 75], [77, 71], [77, 76], [77, 78], [79, 78], [79, 80], [80, 81], [81, 72], [81, 82], [82, 83], [83, 92], [84, 83], [85, 73], [85, 84], [85, 86], [86,

In [6]:
mlp_edge_feature = tf.keras.models.load_model("models/mlp_edge_features.keras")
dnn = tf.keras.models.load_model("models/dnn.keras")
mlp = tf.keras.models.load_model("models/mlp.keras")

2023-11-27 00:47:15.952479: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-11-27 00:47:17.175433: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-27 00:47:17.179321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 Ti computeCapability: 7.5
coreClock: 1.485GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 178.84GiB/s
2023-11-27 00:47:17.179616: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-11-27 00:47:17.179711: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynam

In [7]:
SS = StandardScaler()
num_qubits = 7
count_or = 0
count_nr = 0
count_mlp = 0
count_mlp_edge = 0
for qc_i in tqdm(range(len(test_dataset[:5]))):
    qc = test_dataset[qc_i]
    data = datetime.today() - timedelta(days=random.randint(1,150))
    backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
    backend = backends[np.random.randint(0,3)]
    l = add_line(qc,backend, optimization_level=3, refresh=True, show= False, datatime=data)
    
    d={}
    for i in range(len(l[0])):
        d[str(l[0][i])] = l[1][i]
    df = pd.DataFrame(d, index=[0])
    df = clear_dataset(df, 7)
    last_num_qubits = len(df.columns)-num_qubits

    x = SS.fit_transform(df.iloc[:, 2:last_num_qubits].values)
    labels = ((df.iloc[:, last_num_qubits:].values)[0]).tolist()
    
    predicted = dnn.predict(x)
    
    pred_or = np.array(pred_layout(predicted, num_qubits))[0].tolist()
    pred_nr = np.array(pred_layout_diff_elem(predicted,num_qubits))[0].tolist()
    print(pred_or, pred_nr, labels)
    if(pred_or == labels):
        count_or = count_or + 1
    if(pred_nr == labels):
        count_nr = count_nr + 1
    
    mlp_pred = get_labels(np.reshape(mlp.predict(x),(7,7)))
    print(mlp_pred)
    if mlp_pred == labels:
        count_mlp = count_mlp + 1
        
        
    x = SS.fit_transform(get_graph_features(df))
    mlp_edges_pred = get_labels(np.reshape(mlp_edge_feature.predict(x),(7,7)))
    print(mlp_edges_pred)
    if mlp_edges_pred == labels:
        count_mlp_edge = count_mlp_edge + 1
        
print("DNN accuracy without repair operator:", count_or/len(test_dataset))
print("DNN accuracy with repair operator:", count_nr/len(test_dataset))
print("MLP accuracy:", count_mlp/len(test_dataset))
print("MLP with edge features accuracy:", count_mlp_edge/len(test_dataset))

  0%|                                                                                                                     | 0/5 [00:00<?, ?it/s]

2023-07-06 00:47:19.848567
Backend topology: 2023-07-06 00:47:19.848567
Depth: [6, 6, 6, 6, 6, 6]
na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



 20%|█████████████████████▊                                                                                       | 1/5 [00:02<00:09,  2.49s/it]

[2, 1, 0, 3, 4, 5, 6]
2023-07-24 00:47:22.337632
Backend topology: 2023-07-24 00:47:22.337632
Depth: [6, 6, 6, 6, 6, 6]
na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



 40%|███████████████████████████████████████████▌                                                                 | 2/5 [00:04<00:05,  2.00s/it]

...

[2, 1, 0, 3, 4, 5, 6]
2023-08-20 00:47:25.680667
Backend topology: 2023-08-20 00:47:25.680667
Depth: [6, 6, 6, 6, 6, 6]
na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



 80%|███████████████████████████████████████████████████████████████████████████████████████▏                     | 4/5 [00:09<00:02,  2.43s/it]

[2, 1, 0, 3, 4, 5, 6]
2023-09-20 00:47:28.997476
Backend topology: 2023-09-20 00:47:28.997476
Depth: [6, 6, 6, 6, 6, 6]
na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.15s/it]

[2, 1, 0, 3, 4, 5, 6]
DNN accuracy without repair operator: 0.0
DNN accuracy with repair operator: 0.0
MLP accuracy: 0.0
MLP with edge features accuracy: 0.0
